In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
# from sklearn.preprocessing import Imputer
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
# import tflearn
import tensorflow as tf
import seaborn
import warnings
warnings.filterwarnings('ignore')

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

In [37]:
train = pd.read_csv('train_t0.csv')
label = pd.read_csv('train_t1.csv')
print(train.shape)
print(label.shape)
# test = pd.read_csv('tessv')
data = pd.concat([train],ignore_index=True)
data = data.drop("ID",1)
data.head()

(150, 596)
(150, 596)


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f556,f557,f558,f559,f560,f561,f562,f563,f564,f565,f566,f567,f568,f569,f570,f571,f572,f573,f574,f575,f576,f577,f578,f579,f580,f581,f582,f583,f584,f585,f586,f587,f588,f589,f590,f591,f592,f593,f594,f595
0,0.193382,0.096121,0.289502,0,0.193382,0.096121,0.273247,0.466629,0.177126,0.273247,0.383924,0.577305,0.287803,0.383924,0.110677,0.279031,0.472412,0.182910,0.279031,0.005784,0.104893,0.110154,0.303536,0.014033,0.110154,0.163093,0.273770,0.168877,0.656734,0.850116,0.560613,0.656734,0.383487,0.272810,0.377703,0.546580,0.048374,0.145008,0.144495,0.048374,...,0.126646,0.048892,0.193999,0.091083,0.063588,0.063588,0.015484,0.208866,0.080636,0.015484,0.257763,0.368439,0.263546,0.094670,0.641250,0.063858,0.251580,0.445395,0.343711,0.533262,0.209793,0.433611,0.070879,0.055528,0.217489,0.052125,0.199823,0.115555,0.533486,0.012033,0.110908,0.193465,0.186630,0.174750,0.096996,0.242103,0.042979,0.015484,0.015484,0.048103
1,0.014410,0.064584,0.050174,0,0.014410,0.064584,0.115367,0.100957,0.050783,0.115367,0.280751,0.266341,0.216167,0.280751,0.165384,0.133898,0.119488,0.069314,0.133898,0.018531,0.146853,0.114678,0.100268,0.050094,0.114678,0.000689,0.166073,0.019220,0.131172,0.116761,0.066588,0.131172,0.015805,0.149580,0.002727,0.016494,0.065569,0.051158,0.000985,0.065569,...,0.131344,0.142796,0.153824,0.143518,0.229512,0.229512,0.013585,0.027995,0.078169,0.013585,0.128952,0.294336,0.147483,0.128263,0.144757,0.079154,0.139870,0.143458,0.182415,0.120655,0.185169,0.062271,0.096190,0.112874,0.141057,0.152346,0.014494,0.144452,0.025750,0.086677,0.101471,0.028694,0.187073,0.111753,0.100301,0.089273,0.099579,0.013585,0.013585,0.243097
2,0.070879,0.077238,0.148117,0,0.070879,0.077238,0.162528,0.233407,0.085291,0.162528,0.086521,0.157399,0.009283,0.086521,0.076008,0.154000,0.224879,0.076762,0.154000,0.008529,0.067479,0.149078,0.219956,0.071840,0.149078,0.013451,0.062557,0.004922,0.197996,0.268875,0.120758,0.197996,0.035468,0.111476,0.043997,0.048919,0.074722,0.145600,0.002516,0.074722,...,0.012306,0.046075,0.071845,0.042282,0.172846,0.172846,0.046046,0.116925,0.031192,0.046046,0.116482,0.040474,0.107954,0.103031,0.151950,0.028675,0.122083,0.183384,0.183365,0.068136,0.083248,0.091681,0.087642,0.117377,0.190595,0.080120,0.082894,0.108819,0.072999,0.055021,0.085059,0.160802,0.142508,0.114494,0.080725,0.054955,0.084518,0.046046,0.046046,0.126800
3,0.012150,0.053557,0.065708,0,0.012150,0.053557,0.135480,0.147630,0.081922,0.135480,0.065623,0.077774,0.012066,0.065623,0.069856,0.150656,0.162806,0.097099,0.150656,0.015176,0.085032,0.105734,0.117884,0.052177,0.105734,0.029746,0.040111,0.044922,0.126453,0.138604,0.072896,0.126453,0.009026,0.060830,0.024202,0.020719,0.071057,0.058907,0.124615,0.071057,...,0.124101,0.156046,0.150542,0.138081,0.225818,0.225818,0.007430,0.019580,0.046128,0.007430,0.128050,0.058194,0.143226,0.098304,0.119024,0.078487,0.101074,0.116838,0.134783,0.144918,0.126458,0.099879,0.075145,0.101797,0.167031,0.028526,0.083871,0.105492,0.008554,0.062910,0.066011,0.034460,0.122039,0.094287,0.062342,0.067846,0.080307,0.007430,0.007430,0.218388
4,0.035510,0.072512,0.037002,0,0.035510,0.072512,0.056819,0.021309,0.015693,0.056819,0.092735,0.057225,0.020223,0.092735,0.035916,0.154670,0.119161,0.082158,0.154670,0.097851,0.061935,0.147432,0.111922,0.074919,0.147432,0.090612,0.054697,0.007239,0.216227,0.180718,0.143715,0.216227,0.159408,0.123492,0.061557,0.068796,0.038396,0.002886,0.034116,0.038396,...,0.136243,0.151124,0.122952,0.177226,0.250665,0.250665,0.017878,0.017631,0.054634,0.017878,0.038941,0.074857,0.136792,0.129553,0.198349,0.020518,0.154999,0.109175,0.104394,0.120631,0.163377,0.066896,0.080816,0.108183,0.137156,0.111283,0.098557,0.122693,0.014264,0.014730,0.098395,0.117918,0.121873,0.096543,0.081662,0.109835,0.055561,0.017878,0.017878,0.232787


In [46]:
label = label.drop("ID",1)
label.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f556,f557,f558,f559,f560,f561,f562,f563,f564,f565,f566,f567,f568,f569,f570,f571,f572,f573,f574,f575,f576,f577,f578,f579,f580,f581,f582,f583,f584,f585,f586,f587,f588,f589,f590,f591,f592,f593,f594,f595
0,0.100531,0.032178,0.132709,0,0.100531,0.032178,0.155462,0.255994,0.123285,0.155462,0.346908,0.447440,0.314730,0.346908,0.191446,0.144941,0.245472,0.112763,0.144941,0.010521,0.201967,0.111707,0.212238,0.079529,0.111707,0.043756,0.235201,0.033234,0.113329,0.213860,0.081151,0.113329,0.042133,0.233579,0.031612,0.001622,0.006050,0.106581,0.026128,0.006050,...,0.008971,0.065154,0.060051,0.078424,0.162326,0.162326,0.055889,0.044643,0.088066,0.055889,0.211351,0.402797,0.200830,0.167595,0.169218,0.061939,0.196242,0.228053,0.214949,0.199267,0.341571,0.170295,0.147523,0.154134,0.232804,0.179231,0.014141,0.200908,0.054104,0.130524,0.173879,0.241906,0.198942,0.209243,0.153060,0.158163,0.139791,0.055889,0.055889,0.218214
1,0.014826,0.071366,0.056540,0,0.014826,0.071366,0.125061,0.110234,0.053695,0.125061,0.253270,0.238443,0.181904,0.253270,0.128209,0.166416,0.151589,0.095049,0.166416,0.041355,0.086854,0.113256,0.098430,0.041890,0.113256,0.011805,0.140014,0.053159,0.210117,0.195291,0.138751,0.210117,0.085056,0.043153,0.043702,0.096861,0.039016,0.024189,0.032350,0.039016,...,0.101029,0.102435,0.130848,0.165909,0.213594,0.213594,0.008621,0.023447,0.079987,0.008621,0.133682,0.261891,0.175036,0.121877,0.218738,0.047636,0.126726,0.130521,0.147347,0.221152,0.160091,0.076162,0.064230,0.115305,0.140508,0.144069,0.009418,0.138678,0.043492,0.063972,0.064415,0.077555,0.153872,0.121185,0.119779,0.091366,0.056305,0.008621,0.008621,0.222214
2,0.097006,0.099051,0.196056,0,0.097006,0.099051,0.197889,0.294895,0.098839,0.197889,0.323073,0.420079,0.224022,0.323073,0.125184,0.186627,0.283633,0.087576,0.186627,0.011262,0.136446,0.130447,0.227452,0.031396,0.130447,0.067443,0.192626,0.056181,0.218066,0.315072,0.119016,0.218066,0.020177,0.105007,0.031439,0.087620,0.038044,0.135050,0.061006,0.038044,...,0.000494,0.006360,0.006158,0.033650,0.127929,0.127929,0.043688,0.140693,0.055363,0.043688,0.154202,0.279385,0.142940,0.086759,0.174379,0.005643,0.102093,0.144752,0.174812,0.087941,0.074424,0.089609,0.003455,0.147541,0.054193,0.069555,0.084488,0.120292,0.035471,0.026014,0.075619,0.019626,0.117394,0.084735,0.090601,0.078083,0.117891,0.043688,0.043688,0.084241
3,0.024989,0.050223,0.075212,0,0.024989,0.050223,0.126632,0.151621,0.076409,0.126632,0.076875,0.101864,0.026652,0.076875,0.049757,0.156209,0.181198,0.105986,0.156209,0.029577,0.079334,0.114409,0.139399,0.064187,0.114409,0.012222,0.037534,0.041800,0.137521,0.162510,0.087298,0.137521,0.010889,0.060646,0.018688,0.023111,0.025548,0.000558,0.075770,0.025548,...,0.275644,0.318238,0.310798,0.261226,0.385062,0.385062,0.024101,0.049090,0.026121,0.024101,0.102531,0.052774,0.132108,0.090308,0.113420,0.049648,0.087579,0.126865,0.111339,0.166021,0.098746,0.102437,0.038643,0.076978,0.151718,0.021442,0.111813,0.082358,0.045167,0.046630,0.049152,0.004887,0.117996,0.085317,0.042723,0.050163,0.099735,0.024101,0.024101,0.360961
4,0.031143,0.061256,0.030114,0,0.031143,0.061256,0.048167,0.017024,0.013089,0.048167,0.134764,0.103621,0.073507,0.134764,0.086597,0.141375,0.110232,0.080119,0.141375,0.093208,0.006611,0.126412,0.095269,0.065156,0.126412,0.078245,0.008352,0.014963,0.195353,0.164210,0.134097,0.195353,0.147186,0.060590,0.053978,0.068941,0.030095,0.001048,0.031162,0.030095,...,0.133218,0.142490,0.107539,0.149699,0.224556,0.224556,0.024287,0.055429,0.085543,0.024287,0.072454,0.159050,0.165662,0.150699,0.219640,0.054381,0.171454,0.171289,0.169075,0.168031,0.223654,0.136651,0.120110,0.173127,0.261584,0.180769,0.076161,0.159417,0.056674,0.103605,0.145995,0.142238,0.250164,0.115625,0.106353,0.141304,0.099144,0.024287,0.024287,0.248843


In [9]:
print(train.shape)

(150, 596)


In [38]:
pca = PCA(whiten=True)
pca.fit(data)
variance = pd.DataFrame(pca.explained_variance_ratio_)
np.cumsum(pca.explained_variance_ratio_)

array([0.224933  , 0.34055914, 0.41194   , 0.47406611, 0.5260937 ,
       0.5634532 , 0.59881892, 0.63200694, 0.66215058, 0.6887922 ,
       0.71019499, 0.73016214, 0.74716108, 0.7630392 , 0.77814312,
       0.79256131, 0.80501491, 0.81709319, 0.82857371, 0.83846154,
       0.84797267, 0.85667559, 0.86500742, 0.87282312, 0.88008841,
       0.88682917, 0.89270303, 0.89848133, 0.90399686, 0.90902141,
       0.9139036 , 0.91815655, 0.9222748 , 0.92612756, 0.92990596,
       0.93321069, 0.93640118, 0.93955865, 0.94254379, 0.94529901,
       0.94791718, 0.95033645, 0.95258321, 0.95462654, 0.95656349,
       0.95828346, 0.95995984, 0.961577  , 0.96304941, 0.96437739,
       0.96568436, 0.96695246, 0.96817495, 0.96932601, 0.97040638,
       0.97143337, 0.97244976, 0.97336541, 0.97427072, 0.97510283,
       0.97589549, 0.97667951, 0.97743632, 0.97818272, 0.97890677,
       0.97959174, 0.98027202, 0.98093661, 0.98158054, 0.98219442,
       0.9827933 , 0.98337521, 0.98393243, 0.98446685, 0.98498

In [55]:
pca = PCA(n_components=50,whiten=True)
pca = pca.fit(data)
dataPCA = pca.transform(data)
print(dataPCA.shape)
print(label.shape)

(150, 50)
(150, 595)


In [64]:
# R2 Score
from sklearn.multioutput import MultiOutputRegressor

def lets_try(train,labels):
    results={}
    test = train[-15:]
    test_labels = labels[-15:]
    train = train[:-15]
    train_labels = labels[:-15]
    print(train.shape)
    print(test.shape)
    print(train_labels.shape)
    print(test_labels.shape)
    def test_model(clf):
        
        # cv = KFold(n_splits=5,shuffle=True,random_state=45)
        # r2 = make_scorer(mean_squared_error)
        # r2_val_score = cross_val_score(clf, train, labels, cv=cv,scoring=r2)
        # scores=[r2_val_score.mean()]
        
        clf.fit(train, train_labels)
        test_predict = clf.predict(test)
        scores = mean_squared_error(test_labels, test_predict)

        return [scores]

    clf = linear_model.LinearRegression()
    print(f"Linear Regression = {test_model(clf)}")
    
    clf = linear_model.Ridge()
    print(f"Ridge = {test_model(clf)}")
    
    # clf = linear_model.BayesianRidge()
    # print(f"BayesianRidge = {test_model(clf)}")
    
    # clf = linear_model.HuberRegressor()
    # print(f"HuberRegressor= {test_model(clf)}")
    
    clf = linear_model.Lasso(alpha=1e-4)
    print(f"Lasso = {test_model(clf)}")
    
    clf = BaggingRegressor()
    print(f"BaggingRegressor = {test_model(clf)}")
    
    clf = RandomForestRegressor()
    print(f"RandomForestRegressor= {test_model(clf)}")
    
    # clf = MultiOutputRegressor(AdaBoostRegressor())
    # print(f"AdaBoostRegressor = {test_model(clf)}")
    
    clf = MultiOutputRegressor(svm.SVR())
    print(f"SVM RBF = {test_model(clf)}")
    
    clf = MultiOutputRegressor(svm.SVR(kernel="linear"))
    print(f"SVM Linear = {test_model(clf)}")
    
    # results = pd.DataFrame.from_dict(results,orient='index')
    # results.columns=["MSE"] 
    # results=results.sort(columns=["MSE"],ascending=False)
    # results.plot(kind="bar",title="Model Scores")
    # axes = plt.gca()
    # axes.set_ylim([0.5,1])
    # return results

# lets_try(train,train_labels)
print(train_labels.shape)
lets_try(dataPCA, label)

(135, 596)
(135, 50)
(15, 50)
(135, 595)
(15, 595)
Linear Regression = [0.006166049156209831]
Ridge = [0.005934739356022097]
Lasso = [0.005818893242241945]
BaggingRegressor = [0.0028896393852099952]
RandomForestRegressor= [0.0031404242074959585]
SVM RBF = [0.005059513751798852]
SVM Linear = [0.007712577380661428]


In [71]:
clf = BaggingRegressor()
clf.fit(dataPCA, label)


BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=None, verbose=0,
                 warm_start=False)

In [130]:
test = pd.read_csv('test_t0.csv')
test = test.drop("ID",1)
print(test.shape)
test.head()

(80, 595)


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f556,f557,f558,f559,f560,f561,f562,f563,f564,f565,f566,f567,f568,f569,f570,f571,f572,f573,f574,f575,f576,f577,f578,f579,f580,f581,f582,f583,f584,f585,f586,f587,f588,f589,f590,f591,f592,f593,f594,f595
0,0.122661,0.109112,0.231773,0,0.122661,0.109112,0.092518,0.215179,0.016593,0.092518,0.145721,0.268382,0.036609,0.145721,0.053202,0.207891,0.330552,0.098779,0.207891,0.115373,0.062170,0.105800,0.228461,0.003311,0.105800,0.013282,0.039920,0.102091,0.136860,0.259521,0.027748,0.136860,0.044342,0.008861,0.071031,0.031060,0.018215,0.140876,0.090897,0.018215,...,0.047609,0.112625,0.125787,0.124356,0.215772,0.215772,0.004597,0.127258,0.104515,0.004597,0.087922,0.141124,0.203295,0.101204,0.132264,0.013619,0.176267,0.120791,0.196560,0.163665,0.207902,0.065500,0.001600,0.026432,0.187328,0.026848,0.093334,0.115236,0.047054,0.052192,0.083222,0.155292,0.176474,0.163567,0.098551,0.085389,0.086820,0.004597,0.004597,0.211176
1,0.045889,0.077398,0.031509,0,0.045889,0.077398,0.181591,0.135702,0.104193,0.181591,0.079100,0.033211,0.001702,0.079100,0.102491,0.109561,0.063672,0.032163,0.109561,0.072030,0.030461,0.104044,0.058154,0.026646,0.104044,0.077547,0.024944,0.005517,0.181465,0.135576,0.104067,0.181465,0.000126,0.102365,0.071904,0.077421,0.002626,0.043263,0.074772,0.002626,...,0.056347,0.081727,0.075073,0.080828,0.177357,0.177357,0.002731,0.048621,0.080129,0.002731,0.184322,0.081831,0.112292,0.106775,0.184197,0.005358,0.152657,0.110212,0.131043,0.210307,0.115340,0.139567,0.080000,0.131111,0.134455,0.058544,0.080568,0.159891,0.025726,0.076265,0.087444,0.107125,0.168047,0.123742,0.098362,0.105015,0.099260,0.002731,0.002731,0.180088
2,0.020038,0.026596,0.046635,0,0.020038,0.026596,0.114418,0.134456,0.087822,0.114418,0.180229,0.200267,0.153633,0.180229,0.065811,0.177262,0.197300,0.150666,0.177262,0.062844,0.002967,0.136511,0.156549,0.109914,0.136511,0.022092,0.043718,0.040751,0.244241,0.264279,0.217645,0.244241,0.129823,0.064012,0.066979,0.107731,0.007501,0.012537,0.034097,0.007501,...,0.139746,0.083844,0.133137,0.168949,0.236086,0.236086,0.090372,0.070334,0.116968,0.090372,0.204790,0.270601,0.267634,0.226883,0.334613,0.082871,0.265359,0.247751,0.239643,0.292382,0.241814,0.268363,0.193135,0.160032,0.205748,0.116001,0.050774,0.219221,0.058565,0.132854,0.189319,0.125778,0.253501,0.186712,0.242613,0.193321,0.157509,0.090372,0.090372,0.326458
3,0.015838,0.073108,0.057270,0,0.015838,0.073108,0.112882,0.097044,0.039774,0.112882,0.281027,0.265189,0.207919,0.281027,0.168146,0.077977,0.062139,0.004869,0.077977,0.034905,0.203050,0.123877,0.108039,0.050769,0.123877,0.010995,0.157150,0.045900,0.151164,0.135326,0.078056,0.151164,0.038282,0.129863,0.073187,0.027287,0.014713,0.001125,0.058395,0.014713,...,0.135869,0.142534,0.112490,0.172141,0.237667,0.237667,0.002926,0.012912,0.070182,0.002926,0.109955,0.278101,0.075051,0.120951,0.148238,0.011787,0.148583,0.113695,0.114933,0.205219,0.170457,0.113402,0.075274,0.058549,0.129987,0.145470,0.050776,0.101857,0.031142,0.069858,0.072407,0.051349,0.140081,0.098872,0.092207,0.122251,0.062600,0.002926,0.002926,0.234741
4,0.007235,0.048022,0.040787,0,0.007235,0.048022,0.135871,0.128635,0.087848,0.135871,0.314094,0.306859,0.266072,0.314094,0.178223,0.118140,0.110905,0.070118,0.118140,0.017730,0.195954,0.093777,0.086542,0.045755,0.093777,0.042093,0.220317,0.024363,0.204410,0.197175,0.156388,0.204410,0.068540,0.109684,0.086270,0.110633,0.017383,0.024618,0.065405,0.017383,...,0.073855,0.081441,0.093111,0.152142,0.215760,0.215760,0.002288,0.004948,0.045735,0.002288,0.133583,0.311806,0.115853,0.091490,0.202123,0.019670,0.119562,0.102787,0.090560,0.267360,0.176748,0.144538,0.066852,0.085370,0.093304,0.139875,0.129553,0.138968,0.051438,0.047755,0.057658,0.070526,0.123818,0.139617,0.132032,0.120361,0.061331,0.002288,0.002288,0.213473


In [157]:
test = pca.transform(test)
prediction = clf.predict(test)

ValueError: ignored

In [158]:
print(prediction.shape)
df = pd.DataFrame(prediction)
df.head()

(80, 595)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594
0,0.061191,0.069773,0.100056,0.0,0.061191,0.069773,0.120664,0.135496,0.059870,0.120664,0.227928,0.231755,0.160337,0.227928,0.110956,0.131565,0.141528,0.076725,0.131565,0.043175,0.109121,0.117622,0.123289,0.060449,0.117622,0.043911,0.111157,0.031649,0.198721,0.206080,0.131130,0.198721,0.078057,0.091710,0.070446,0.082791,0.051093,0.083781,0.081864,0.051093,...,0.076976,0.101069,0.107085,0.106938,0.215764,0.215764,0.030977,0.082613,0.089364,0.030977,0.139579,0.246842,0.150479,0.136536,0.217636,0.062679,0.159866,0.162779,0.164109,0.186849,0.189360,0.168623,0.091256,0.151026,0.199383,0.142839,0.119177,0.147775,0.053008,0.091477,0.113772,0.137515,0.180175,0.157702,0.133609,0.127593,0.127740,0.030977,0.030977,0.234678
1,0.033321,0.063242,0.075787,0.0,0.033321,0.063242,0.125789,0.117529,0.071123,0.125789,0.197037,0.191868,0.147128,0.197037,0.100103,0.138841,0.130581,0.075599,0.138841,0.050862,0.091326,0.119316,0.114157,0.056073,0.119316,0.031671,0.101574,0.041361,0.172832,0.164572,0.119746,0.172832,0.048624,0.059732,0.064400,0.065150,0.038848,0.039315,0.044569,0.038848,...,0.091768,0.107706,0.101895,0.121103,0.204992,0.204992,0.049087,0.061473,0.099850,0.049087,0.158369,0.229617,0.171422,0.151896,0.205412,0.074795,0.166880,0.168897,0.179105,0.216456,0.161271,0.147585,0.095074,0.151313,0.192701,0.135320,0.062774,0.160402,0.057353,0.102579,0.108577,0.091379,0.184160,0.145804,0.133975,0.137316,0.116469,0.049087,0.049087,0.237572
2,0.049136,0.098683,0.082939,0.0,0.049136,0.098683,0.111037,0.106357,0.042058,0.111037,0.256218,0.239227,0.157535,0.256218,0.156209,0.135236,0.118246,0.050777,0.135236,0.056052,0.135697,0.113027,0.099242,0.028464,0.113027,0.046402,0.153839,0.040784,0.229585,0.212595,0.130903,0.229585,0.118549,0.069618,0.099200,0.116558,0.036031,0.059276,0.107577,0.036031,...,0.145069,0.163035,0.168360,0.167826,0.270541,0.270541,0.033620,0.064268,0.129585,0.033620,0.141939,0.287121,0.166139,0.143930,0.260488,0.057240,0.175739,0.153453,0.182614,0.252735,0.234901,0.160419,0.081080,0.183171,0.255777,0.136256,0.047490,0.179513,0.047582,0.098498,0.131737,0.102939,0.205596,0.156374,0.138409,0.133083,0.133618,0.033620,0.033620,0.301443
3,0.031443,0.081229,0.060299,0.0,0.031443,0.081229,0.125889,0.104958,0.049882,0.125889,0.245207,0.224277,0.163978,0.245207,0.129204,0.132688,0.111758,0.056968,0.132688,0.033799,0.118012,0.122749,0.101818,0.046744,0.122749,0.008972,0.126598,0.027602,0.182832,0.161902,0.101603,0.182832,0.064703,0.085757,0.050144,0.062098,0.053179,0.035509,0.034103,0.053179,...,0.094718,0.108879,0.130633,0.125606,0.207155,0.207155,0.030214,0.049850,0.100684,0.030214,0.145343,0.264662,0.152143,0.142203,0.202286,0.072634,0.154415,0.154221,0.166982,0.200675,0.176652,0.121940,0.074823,0.110906,0.187563,0.124042,0.065879,0.135397,0.077134,0.085023,0.090911,0.083577,0.172182,0.140055,0.129187,0.110687,0.112006,0.030214,0.030214,0.226610
4,0.032790,0.076186,0.076123,0.0,0.032790,0.076186,0.139230,0.144752,0.084616,0.139230,0.279849,0.285371,0.225235,0.279849,0.140619,0.146698,0.152314,0.104802,0.146698,0.063402,0.136481,0.122704,0.128226,0.069898,0.122704,0.033334,0.157145,0.050501,0.172981,0.178503,0.120894,0.172981,0.073674,0.106868,0.055412,0.067624,0.031973,0.052431,0.075301,0.031973,...,0.131004,0.130524,0.149030,0.148690,0.256656,0.256656,0.047922,0.050830,0.118226,0.047922,0.177736,0.318355,0.185205,0.161210,0.211487,0.066122,0.200252,0.150367,0.198834,0.258725,0.237308,0.191463,0.117014,0.154475,0.201439,0.156890,0.063483,0.177274,0.085786,0.092861,0.130106,0.125953,0.196681,0.164158,0.164638,0.146132,0.146472,0.047922,0.047922,0.295162


In [159]:
batu_flat = df.values.flatten()
print(batu_flat.shape)

(47600,)


In [160]:
meltedDF = df.to_numpy().flatten()
print(meltedDF.shape)

(47600,)


In [161]:
anan = pd.DataFrame(meltedDF)
anan.head()

,0
0,0.061191
1,0.069773
2,0.100056
3,0.000000
4,0.061191


In [151]:
# anan.insert(0, "ID", np.arange(1, anan.shape[0]+1))

In [162]:
anan.head()

,0
0,0.061191
1,0.069773
2,0.100056
3,0.000000
4,0.061191


In [163]:
anan.tail()

,0
47595,0.134184
47596,0.134456
47597,0.037313
47598,0.037313
47599,0.310460


In [164]:
print(anan.shape)

(47600, 1)


In [166]:
baban = anan.rename(columns={0: "predicted"})

In [167]:
baban.head()

,predicted
0,0.061191
1,0.069773
2,0.100056
3,0.000000
4,0.061191


In [169]:
csvdosyasiseysi = baban.to_csv("test_results2.csv")